In [1]:
import sys
sys.path.insert(0, '../ELINA/python_interface/')

import numpy as np
import re
import csv
from elina_box import *
from elina_interval import *
from elina_abstract0 import *
from elina_manager import *
from elina_dimension import *
from elina_scalar import *
from elina_interval import *
from elina_linexpr0 import *
from elina_lincons0 import *
import ctypes
from ctypes.util import find_library
from gurobipy import *
import time
from datetime import datetime
from pprint import pprint
import copy
import warnings

libc = CDLL(find_library('c'))
cstdout = c_void_p.in_dll(libc, 'stdout')

In [2]:
# Import for debugging in jupyter notebook
from IPython.core.debugger import set_trace #TODO remove at end.

In [3]:
class layers:
    def __init__(self):
        self.layertypes = []
        self.weights = []
        self.biases = []
        self.numlayer = 0
        self.ffn_counter = 0
        self.rank = []
        self.use_LP = []
        self.LB_hat = []
        self.UB_hat = []

In [4]:
def parse_bias(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    #return v.reshape((v.size,1))
    return v

def parse_vector(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    return v.reshape((v.size,1))
    #return v
    
def balanced_split(text):
    i = 0
    bal = 0
    start = 0
    result = []
    while i < len(text):
        if text[i] == '[':
            bal += 1
        elif text[i] == ']':
            bal -= 1
        elif text[i] == ',' and bal == 0:
            result.append(text[start:i])
            start = i+1
        i += 1
    if start < i:
        result.append(text[start:i])
    return result

def parse_matrix(text):
    i = 0
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    return np.array([*map(lambda x: parse_vector(x.strip()).flatten(), balanced_split(text[1:-1]))])

def parse_net(text):
    lines = [*filter(lambda x: len(x) != 0, text.split('\n'))]
    i = 0
    res = layers()
    while i < len(lines):
        if lines[i] in ['ReLU', 'Affine']:
            W = parse_matrix(lines[i+1])
            b = parse_bias(lines[i+2])
            res.layertypes.append(lines[i])
            res.weights.append(W)
            res.biases.append(b)
            res.numlayer+= 1
            res.rank.append(np.zeros((W.shape[0],1)))
            res.use_LP.append(np.full((W.shape[0],1), False))
            res.LB_hat.append(np.full((W.shape[0],1), np.nan))
            res.UB_hat.append(np.full((W.shape[0],1), np.nan))
            i += 3
        else:
            raise Exception('parse error: '+lines[i])
    return res

def parse_spec(text):
    text = text.replace("[", "")
    text = text.replace("]", "")
    with open('dummy', 'w') as my_file:
        my_file.write(text)
    data = np.genfromtxt('dummy', delimiter=',',dtype=np.double)
    low = copy.deepcopy(data[:,0])
    high = copy.deepcopy(data[:,1])
    return low,high

In [5]:
def get_perturbed_image(x, epsilon):
    image = x[1:len(x)]
    num_pixels = len(image)
    LB_N0 = image - epsilon
    UB_N0 = image + epsilon
     
    for i in range(num_pixels):
        if(LB_N0[i] < 0):
            LB_N0[i] = 0
        if(UB_N0[i] > 1):
            UB_N0[i] = 1
    return LB_N0, UB_N0

In [6]:
def generate_linexpr0(weights, bias, size):
    linexpr0 = elina_linexpr0_alloc(ElinaLinexprDiscr.ELINA_LINEXPR_DENSE, size)
    cst = pointer(linexpr0.contents.cst)
    elina_scalar_set_double(cst.contents.val.scalar, bias)
    for i in range(size):
        elina_linexpr0_set_coeff_scalar_double(linexpr0,i,weights[i])
    return linexpr0

In [7]:
def analyze(nn, LB_N0, UB_N0, label):   
    num_pixels = len(LB_N0)
    nn.ffn_counter = 0
    numlayer = nn.numlayer 
    man = elina_box_manager_alloc()
    itv = elina_interval_array_alloc(num_pixels)
    for i in range(num_pixels):
        elina_interval_set_double(itv[i],LB_N0[i],UB_N0[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_pixels, itv)
    elina_interval_array_free(itv,num_pixels)
    for layerno in range(numlayer):
        if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
           weights = nn.weights[nn.ffn_counter]
           biases = nn.biases[nn.ffn_counter]
           dims = elina_abstract0_dimension(man,element)
           num_in_pixels = dims.intdim + dims.realdim
           num_out_pixels = len(weights)

           dimadd = elina_dimchange_alloc(0,num_out_pixels)    
           for i in range(num_out_pixels):
               dimadd.contents.dim[i] = num_in_pixels
           elina_abstract0_add_dimensions(man, True, element, dimadd, False)
           elina_dimchange_free(dimadd)
           np.ascontiguousarray(weights, dtype=np.double)
           np.ascontiguousarray(biases, dtype=np.double)
           var = num_in_pixels
           # handle affine layer
           for i in range(num_out_pixels):
               tdim= ElinaDim(var)
               linexpr0 = generate_linexpr0(weights[i],biases[i],num_in_pixels)
               element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
               var+=1
           dimrem = elina_dimchange_alloc(0,num_in_pixels)
           for i in range(num_in_pixels):
               dimrem.contents.dim[i] = i
           elina_abstract0_remove_dimensions(man, True, element, dimrem)
           elina_dimchange_free(dimrem)
           # handle ReLU layer 
           if(nn.layertypes[layerno]=='ReLU'):
              element = relu_box_layerwise(man,True,element,0, num_out_pixels)
           nn.ffn_counter+=1 

        else:
           print(' net type not supported')
   
    dims = elina_abstract0_dimension(man,element)
    output_size = dims.intdim + dims.realdim
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)

           
    # if epsilon is zero, try to classify else verify robustness 
    
    verified_flag = True
    predicted_label = 0
    if(LB_N0[0]==UB_N0[0]):
        for i in range(output_size):
            inf = bounds[i].contents.inf.contents.val.dbl
            flag = True
            for j in range(output_size):
                if(j!=i):
                   sup = bounds[j].contents.sup.contents.val.dbl
                   if(inf<=sup):
                      flag = False
                      break
            if(flag):
                predicted_label = i
                break    
    else:
        inf = bounds[label].contents.inf.contents.val.dbl
        for j in range(output_size):
            if(j!=label):
                sup = bounds[j].contents.sup.contents.val.dbl
                if(inf<=sup):
                    predicted_label = label
                    verified_flag = False
                    break

    elina_interval_array_free(bounds,output_size)
    elina_abstract0_free(man,element)
    elina_manager_free(man)        
    return predicted_label, verified_flag

## Define operations on abstract domain using linear approximations

In [8]:
def add_hidden_constraint(model, layerno, z, z_hat, weights, biases):
    """
    This function computes “which side” of the ReLU the pre-ReLU activations lies on.
    INPUT:
        - model: gurobi model
        - layerno: layer number from which z_hat belong
        - z: gurobi variables for hidden layer input
        - z_hat: gurobi variables for hidden layer output
        - weights: weights for the hidden layer
        - bias: bias in the hidden layer
    OUTPUT:
        - model: gurobi model with new hidden constrains
   """
    # Sanity check!
    assert len(z) == weights.shape[1]
    assert len(z_hat) == weights.shape[0]
    
    # add constraint to model
    for i_out in range(len(z_hat)):
        constr = LinExpr() + np.asscalar(biases[i_out])
        for s in range(len(z)):
            constr += z[s] * np.asscalar(weights[i_out, s])

        model.addConstr(z_hat[i_out] == constr, \
                    name="hidden_constr_" + str(layerno) + "_" + str(i_out))
    
    model.update()
    return model

In [9]:
def add_relu_activation_constraint(model, layerno, z_hat, z, LB, UB):
    """
    This function computes “which side” of the ReLU the pre-ReLU activations lies on.
    INPUT:
        - model: gurobi model
        - layerno: layer number from which z_hat belong
        - z_hat: gurobi variables for pre-relu input
        - z: gurobi variables for relu output
        - LB: lower bound of inputs to a relu layer
        - UB: upper bound of inputs to a relu layer
    OUTPUT:
        - model: gurobi model with new ReLU constrains
   """
    # Sanity check!
    assert len(z) == len(UB)
    
    # iterate over each pre-relu neuron activation
    for j in range(len(UB)):
        u = np.asscalar(UB[j])
        l = np.asscalar(LB[j])

        if u <= 0:
            model.addConstr(z[j] == 0, \
                        name="relu_constr_deac_" + str(layerno) + "_" + str(j))
        elif l > 0:
            model.addConstr(z[j] == z_hat[j], \
                        name="relu_constr_deac_" + str(layerno) + "_" + str(j))
        else:
            alpha = u/(u - l)
            model.addConstr(z[j] >= 0 , \
                         name="relu_const_ambi_pos_" + str(layerno) + "_" + str(j))
            model.addConstr(z[j] >= z_hat[j], \
                         name="relu_const_ambi_hid_" + str(layerno) + "_" + str(j))
            model.addConstr(z[j] <= alpha * (z_hat[j] - l), \
                         name="relu_const_ambi_lin_" + str(layerno) + "_" + str(j))
    model.update()
    return model

In [10]:
def call_linear_solver(model, z_hat, lb_only=False, ub_only=False):
    """
    This function computes lower and upper bound for given objective function and model
    INPUT:
        - model: gurobi model
        - z_hat: gurobi variable to optimize for
    OUTPUT:
        - LB: lower bound of variable
        - UB: upper bound of variable
   """
    # Sanity
    assert not lb_only*ub_only
    LB, UB = None, None
    
    if not ub_only:
        # Find Lower Bound
        model.setObjective(z_hat, GRB.MINIMIZE)
        model.update()
        model.optimize()

        if model.status == GRB.Status.OPTIMAL:
            LB = model.objVal
        else:
            raise(RuntimeError('[Min] Error. Not Able to retrieve bound. Gurobi Model. Not Optimal.'))

        # reset model 
        model.reset()
    
    if not lb_only:
        # Find Upper Bound
        model.setObjective(z_hat, GRB.MAXIMIZE)
        model.update()
        model.optimize()

        if model.status == GRB.Status.OPTIMAL:
            UB = model.objVal
        else:
            raise(RuntimeError('[Max] Error. Not Able to retrieve bound. Gurobi Model. Not Optimal.'))

        # reset model 
        model.reset()
    
    return LB, UB

## Define operations on abstract domain using Box approximations

In [11]:
def get_relu_bounds_using_box(man, input_LB, input_UB, num_in_pixels):
    '''
    This function calculates the bounds of a ReLU operation. 
    INPUT:
        - man: pointer to elina manager
        - input_LB: lower bound of the inputs to the ReLU
        - input_UB: upper bound of the inputs to the ReLU
        - num_in_pixels: number of inputs to ReLU
    
    OUTPUT:
        - output_LB: lower bound of the outputs from ReLU layer
        - output_UB: upper bound of the outputs from ReLU layer
        - num_out_pixels: number of outputs of ReLI layer
    '''
    itv = elina_interval_array_alloc(num_in_pixels)

    ## Populate the interval
    for i in range(num_in_pixels):
        elina_interval_set_double(itv[i], input_LB[i], input_UB[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_in_pixels, itv)
    elina_interval_array_free(itv, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle ReLU Layer
    # ------------------------------------------------------------------
    num_out_pixels = num_in_pixels
    
    element = relu_box_layerwise(man, True, element,0, num_in_pixels)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    output_LB = np.zeros((num_out_pixels, 1), float)
    output_UB = np.zeros((num_out_pixels, 1), float)
    for j in range(num_out_pixels):
        output_LB[j] = bounds[j].contents.inf.contents.val.dbl
        output_UB[j] = bounds[j].contents.sup.contents.val.dbl
    
    # free out the memory allocations
    elina_interval_array_free(bounds, num_out_pixels)
    elina_abstract0_free(man, element)
    
    return output_LB, output_UB, num_out_pixels

In [12]:
def get_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels, verbose=False):
    '''
    This function calculates the bounds of a ReLU operation followed by a hidden layer. 
    INPUT:
        - man: pointer to elina manager
        - weights: weights of the hidden layer
        - biases: biases of the hidden layer
        - input_LB: lower bound of the inputs to the hidden layer
        - input_UB: upper bound of the inputs to the hidden layer
        - num_in_pixels: number of inputs to the input layer
    
    OUTPUT:
        - output_LB: lower bound of the outputs from hidden layer
        - output_UB: upper bound of the outputs from hidden layer
        - num_out_pixels: number of outputs of hidden layer
    '''
    itv = elina_interval_array_alloc(num_in_pixels)

    ## Populate the interval
    for i in range(num_in_pixels):
        elina_interval_set_double(itv[i], input_LB[i], input_UB[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_in_pixels, itv)
    elina_interval_array_free(itv, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle Affine Layer
    # ------------------------------------------------------------------

    # calculate number of outputs
    num_out_pixels = len(weights)
    
    if verbose:
        print("[Network] Input pixels: " + str(num_in_pixels))
        print("[Network] Shape of weights: " + str(np.shape(weights)))
        print("[Network] Shape of biases: " + str(np.shape(biases)))
        print("[Network] Out pixels: " + str(num_out_pixels))

    # Create number of neurons in the layer and populate it
    # with the number of inputs to each neuron in the layer
    dimadd = elina_dimchange_alloc(0, num_out_pixels)    
    for i in range(num_out_pixels):
        dimadd.contents.dim[i] = num_in_pixels

    # Add dimensions to an ElinaAbstract0 pointer i.e. element
    elina_abstract0_add_dimensions(man, True, element, dimadd, False)
    elina_dimchange_free(dimadd)

    # Create the linear expression associated each neuron
    var = num_in_pixels
    for i in range(num_out_pixels):
        tdim = ElinaDim(var)
        linexpr0 = generate_linexpr0(weights[i], biases[i], num_in_pixels)
        # Parallel assignment of several dimensions of an ElinaAbstract0 by using an ElinaLinexpr0Array
        element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
        var += 1

    # Pointer to which semantics we want to follow.
    dimrem = elina_dimchange_alloc(0, num_in_pixels)
    for i in range(num_in_pixels):
        dimrem.contents.dim[i] = i
        
    # Remove dimensions from an ElinaAbstract0
    elina_abstract0_remove_dimensions(man, True, element, dimrem)
    elina_dimchange_free(dimrem)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    output_LB = np.zeros((num_out_pixels, 1), float)
    output_UB = np.zeros((num_out_pixels, 1), float)
    for j in range(num_out_pixels):
        output_LB[j] = bounds[j].contents.inf.contents.val.dbl
        output_UB[j] = bounds[j].contents.sup.contents.val.dbl    
    
    # free out the memory allocations
    elina_interval_array_free(bounds, num_out_pixels)
    elina_abstract0_free(man, element)
    
    return output_LB, output_UB, num_out_pixels

# Define function to verify the neural network

In [13]:
def verify_network(LB_N0, UB_N0, LB_NN, UB_NN, label, num_input_pixels = 784, num_out_pixels = 10):
    '''
    This function verifies the network given the bounds of the input layer and the final layer of the network.
    INPUT:
        - LB_N0: lower bounds of the preturbed input image
        - UB_N0: unpper bounds of the preturbed input image
        - LB_NN: lower bounds of the final layer of neural network
        - UB_NN: upper bounds of the final layer of neural network
        - label: true label of the input image
        - num_input_pixels: number of pixels in the input image (for MNIST, default: 784)
        - num_out_pixels: number of neurons in the last layer of the network  (for MNIST, default: 10)
    
    OUTPUT:
        - predicted_label: label predicted by the neural network
        - verified_flag: boolean variable, true if the network is robust to perturbation
    '''
    
    # if epsilon is zero, try to classify else verify robustness 
    verified_flag = True
    predicted_label = 0
    if(LB_N0[0]==UB_N0[0]):
        for i in range(num_out_pixels):
            inf = LB_NN[i]
            flag = True
            for j in range(num_out_pixels):
                if(j!=i):
                    sup = UB_NN[j]
                    if(inf<=sup):
                        flag = False
                        break
            if(flag):
                predicted_label = i
                break    
    else:
        inf = LB_NN[label]
        for j in range(num_out_pixels):
            if(j!=label):
                sup = UB_NN[j]
                if(inf<=sup):
                    predicted_label = label
                    verified_flag = False
                    break

    if(verified_flag):
        print("verified")
    else:
        print("can not be verified")  
        
    return predicted_label, verified_flag

# Functions to perform different analysis

In [14]:
# function to perform box analysis for all layers in the neural network nn
def perform_box_analysis(nn, LB_N0, UB_N0, verbose = False):
    # create a list to store the bounds found through box approximation
    LB_hidden_box_list = []
    UB_hidden_box_list = []

    # create manager for Elina
    man = elina_box_manager_alloc()

    # initialize variables for the network iteration
    numlayer = nn.numlayer 
    nn.ffn_counter = 0

    # for input image
    input_LB = LB_N0.copy()
    input_UB = UB_N0.copy()
    num_in_pixels = len(LB_N0)
    
    if verbose:
        print("Input Layer, size: " + str(len(LB_N0)))
        print('---------------')

    for layerno in range(numlayer):
        if verbose:
            print("Layer Number: " + str(layerno + 1))

        if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
            if verbose:
                print("Layer Type: %s" % nn.layertypes[layerno])

            # read the layer weights and biases
            weights = nn.weights[nn.ffn_counter]
            biases = nn.biases[nn.ffn_counter]
            np.ascontiguousarray(weights, dtype=np.double)
            np.ascontiguousarray(biases, dtype=np.double)

            # ------------------------------------------------------------------
            # Handle Affine Layer
            # ------------------------------------------------------------------
            output_LB, output_UB, num_out_pixels = get_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels, verbose)

            # Add bounds to the list
            LB_hidden_box_list.append(output_LB.copy())
            UB_hidden_box_list.append(output_UB.copy())
            # Prepare variables for next layer
            input_LB = output_LB.copy()
            input_UB = output_UB.copy()
            num_in_pixels = num_out_pixels
            nn.ffn_counter += 1 

            # ------------------------------------------------------------------
            # Handle ReLU Layer
            # ------------------------------------------------------------------
            if(nn.layertypes[layerno] == "ReLU"):
                output_LB, output_UB, num_out_pixels = get_relu_bounds_using_box(man, input_LB, input_UB, num_in_pixels)

            # Prepare variables for next layer
            input_LB = output_LB.copy()
            input_UB = output_UB.copy()

            if verbose:
                print("[OUTPUT] Bounds: ")
                output_LB, output_UB  = output_LB.squeeze(), output_UB.squeeze()
                pprint(np.stack((output_LB, output_UB), axis=1))
            
            if verbose:
                print('---------------')

        else:
            print(' net type not supported')
    if verbose:
        print("Output Layer, size: " + str(len(output_LB)))

    elina_manager_free(man)
    
    # for last layer of the netowork is ReLU
    LB_NN = LB_hidden_box_list[-1].copy()
    UB_NN = UB_hidden_box_list[-1].copy()

    if nn.layertypes[-1] == "ReLU" :
        num_out = len(LB_hidden_box_list[-1])
        for i in range(num_out):
            if LB_hidden_box_list[-1][i] < 0 :
                LB_NN[i] = 0 
            if UB_hidden_box_list[-1][i] < 0 :
                UB_NN[i] = 0 
            
    return LB_hidden_box_list, UB_hidden_box_list, LB_NN.squeeze(), UB_NN.squeeze()

In [15]:
def perform_linear_over_box_approximation(nn, LB_N0, UB_N0, LB_hidden_box_list, UB_hidden_box_list, verbose = False):
    # initialize variables for the network iteration
    numlayer = nn.numlayer 
    nn.ffn_counter = 0
    num_in_pixels = len(LB_N0)
    
    m = get_model()
    
    # We follow the following notations:
    # z_hat_{i} = W_i * z_i + b_i, where i = {1, . . . , k}, and z_hat_{k} = y (ouput of NN)
    # z_i = max(z_hat_{i-1} , 0),  where i = {2, . . . , k}, and z_1 = x (input to NN)

    # for input layer:
    # Variable lower bound: Note that any value less than -1e20 is treated as negative infinity. 
    num_in_pixels = len(LB_N0)
    img_vars = m.addVars(num_in_pixels, lb=LB_N0, ub=UB_N0, \
                         vtype=GRB.CONTINUOUS, name="input_layer")
    
    # for output of each ReLU
    z = []
    z.append(img_vars)
    # for output of each hidden layer
    z_hat = []

    # Create variables for all layers and append to the list 
    m, z, z_hat = add_all_vars(m, numlayer, LB_N0, UB_N0, UB_hidden_box_list)
    m.update()
    
    if verbose: 
        # Sanity check!
        # Size of z should be number of relu activation layers + 1 (for input)
        print("Number of relu layers: {0}".format(len(z)))
        # Size of z_hat should be number of hidden layers
        print("Number of hidden layers: {0}".format(len(z_hat)))
        print("Size of last hidden layer: {0}".format(len(z_hat[-1])))
        print("------------------------------")

    nn.ffn_counter = 0

    # Adding weights constraints for k layers
    for layerno in range(numlayer):
        if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
            # read the layer weights and biases
            weights = nn.weights[nn.ffn_counter]
            biases = nn.biases[nn.ffn_counter]
            np.ascontiguousarray(weights, dtype=np.float)
            np.ascontiguousarray(biases, dtype=np.float)

            # add affine constraint
            add_hidden_constraint(m, layerno, z[layerno], z_hat[layerno], weights, biases)
        
            # update counter for next iteration
            nn.ffn_counter += 1
        else:
            raise("Not a valid layer!")
        
    m.update()

    # Adding relu constraints for (k-1) layers. The loop starts from z_2 since z_1 is input
    for i in range(1, numlayer):        
        # add relu constraint
        if (nn.layertypes[layerno] in ["ReLU"]):
            add_relu_activation_constraint(m, layerno, z_hat[i-1], z[i], LB_hidden_box_list[i-1], UB_hidden_box_list[i-1])

    m.update()

    # storing upper and lower bounds for last layer
    UB = np.zeros_like(nn.biases[-1])
    LB = np.zeros_like(nn.biases[-1])
    numlayer = nn.numlayer 

    # Solving for each neuron in the output layer to collect bounds
    # i.e. z_hat_{-1} where -1 denotes the last array in list
    for i_out in range(len(UB)): 
        LB[i_out], UB[i_out] = call_linear_solver(m, z_hat[-1][i_out])
        
    # for last layer of the netowork is ReLU
    LB_NN = LB
    UB_NN = UB
    
    if nn.layertypes[-1] == "ReLU" :
        num_out = len(UB_NN)

        for i in range(num_out):
            if LB[i] < 0 :
                LB_NN[i] = 0 
            if UB[i] < 0 :
                UB_NN[i] = 0 
                
    return LB_NN.squeeze(), UB_NN.squeeze()

In [16]:
def get_model(single_thread=False):
    """
    Get Gurobi model
    """
    m = Model("LP")
    m.setParam("outputflag", False)

    # disable parallel Gurobi solver
    m.setParam("Method", 1)  # dual simplex
    if single_thread:
        m.setParam("Threads", 1) # only 1 thread
    return m

In [17]:
def add_all_vars(m, numlayer, LB_N0, UB_N0, UB_hidden_box_list, verbose=True):
    """
    Add and create all variables of neural network to gurobi model.
    INPUT:
        - m: Gurobi model
        - numlayer: Number of Layers
        - LB_N0: Lower Bound of perturbed image input
        - UB_N0: Upper Bound of perturbed image input
        - UB_hidden_box_list: List of upper Bounds from box approximation (needed to set upper bound of ReLU outputs)
    OUTPUT:
        - m: Gurobi model with newly added variables
        - z: List of Gurobi variables corresponding to pre-ReLU Layer (hidden)
        - z_hat: List of Gurobi variables corresponding to post-ReLU Layer

    """
    
    # for output of each ReLU
    z = []
    # for output of each hidden layer
    z_hat = []
    
    # Create variables of input image
    num_in_pixels = len(LB_N0)
    img_vars = m.addVars(num_in_pixels, lb=LB_N0, ub=UB_N0, \
                 vtype=GRB.CONTINUOUS, name="input_layer")
    z.append(img_vars)
    
    # Create variables for all layers and append to the list 
    for i in range(numlayer):
        # for layers before the final layer, z_hat and z exists
        if i < (numlayer - 1):

            UB_relu = UB_hidden_box_list[i].squeeze().copy()
            for j in range(len(UB_hidden_box_list[i])):
                bound = UB_hidden_box_list[i][j]
                UB_relu[j] = max(0, bound)
            UB_relu.squeeze() 

            # middle layer, has both z and z hat
            z_hat_hidden = m.addVars(len(UB_hidden_box_list[i]), lb=-np.inf, ub=np.inf, \
                                     vtype=GRB.CONTINUOUS, name="hidden_layer_" + str(i))
            z_relu = m.addVars(len(UB_hidden_box_list[i]), lb=0.0, ub = UB_relu,\
                               vtype=GRB.CONTINUOUS, name="relu_layer_" + str(i))
            # append to the list
            z_hat.append(z_hat_hidden)
            z.append(z_relu)
        # for last layer, only z_hat exists
        else: 
            z_hat_hidden = m.addVars(len(UB_hidden_box_list[i]), lb=-np.inf, ub=np.inf, \
                                     vtype=GRB.CONTINUOUS, name="output_layer") 
            # append to the list
            z_hat.append(z_hat_hidden)

    m.update()
    
    if verbose:
        # Sanity check!
        # Size of z should be number of relu activation layers + 1 (for input)
        print("Number of relu layers: {0}".format(len(z)))
        # Size of z_hat should be number of hidden layers
        print("Number of hidden layers: {0}".format(len(z_hat)))
        print("Size of last hidden layer: {0}".format(len(z_hat[-1])))
    
    return m, z, z_hat

In [18]:
def compute_rank(nn, norm=1, skip_first_layer=True, skip_last_layer=True):
    numlayer = nn.numlayer 
    norms = np.zeros((nn.weights[1].shape[0], numlayer-2)) # TODO use list or similar if first and last layer are also used
    rank_local_idxs = np.zeros((nn.weights[1].shape[0], numlayer-2))
    n_neurons = rank_local_idxs.shape[0] * rank_local_idxs.shape[1]
    
    for layerno in range(numlayer):
        if skip_first_layer and layerno == 0:
            #TODO treat first layer
            continue
        if skip_last_layer and layerno == numlayer-1:
            continue
        if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
            weights = nn.weights[layerno]
            biases = nn.biases[layerno]
#             norms[:, layerno-1] = np.linalg.norm(weights, ord=norm, axis=1) + biases
            norms[:, layerno-1] = np.linalg.norm(weights, ord=norm, axis=0)

        else:
            print(' net type not supported')
    
    # Global rank
    temp = np.argsort(-norms, axis=None)
    rank_idxs = np.empty_like(temp)
    rank_idxs[temp] = np.arange(len(temp))
    rank_idxs = rank_idxs.reshape(nn.weights[1].shape[0], int(len(rank_idxs)/ nn.weights[1].shape[0])) /n_neurons
    
    # Local rank
    temp = np.argsort(-norms, axis=0)
    rank_local_idxs = np.empty_like(temp)
    for layerno in range(numlayer-2):
        rank_local_idxs[temp[:,layerno],layerno] = np.arange(norms.shape[0])
    rank_local_idxs = rank_local_idxs/rank_local_idxs.shape[0]
    
    for layerno in range(numlayer):
        if skip_first_layer and layerno == 0:
            continue
        if skip_last_layer and layerno == numlayer-1:
            continue
        if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
            nn.rank[layerno] = {}
            nn.rank[layerno]['global'] = rank_idxs[:, layerno-1] # TODO Change minus 1 to flexibe offset
            nn.rank[layerno]['local'] = rank_local_idxs[:, layerno-1] # TODO Change minus 1 to flexibe offset
#             nn.use_LP[layerno] = nn.rank[layerno]  < rank_threshold
        else:
            print(' net type not supported')
            
    return

In [46]:
def perform_linear_layerwise(nn, numlayer, LB_N0, UB_N0, lp_list, LB_hidden_box_list, UB_hidden_box_list, 
                             true_label, influence_threshold=[1.0, 0.0] ,verbose=True):
    """
    Get final bounds using linear programming layerwise. If lp_freq > 1 linear bounds are only calculated every 
    lp_freq'th layer.
    INPUT:
        - m: Gurobi model
        - z: List of Gurobi variables corresponding to pre-ReLU Layer (hidden)
        - z_hat: List of Gurobi variables corresponding to post-ReLU Laye
        - nn: Neural Network as defined in initial code (contains layertypes, weights, etc.)
        - numlayer: Number of Layers
        - LB_N0: Lower Bound of perturbed image input
        - UB_N0: Upper Bound of perturbed image input
        - lp_list: Layerno to start solvingbounds by LP. 
        - prob: probability to select a neuron
        - LB_hidden_box_list: List of upper Bounds from box approximation
        - UB_hidden_box_list: List of upper Bounds from box approximation
        - influence_threshold: List of influence threshold where LP is used. First element corresponds 
                                 to threshold of layers in lp_list, second element to other layers.
                                 Therefore, if layer is in lp_list, use LP if rank of Neuron is below 
                                 influence_threshold[0], default is use LP on ALL neurons of lp_list layer.
                                 If layer is not in lp_list, use LP if rank of Neuron is below 
                                 influence_threshold[1], default do NOT use LP on neurons.
                                 
    OUTPUT:
        - LB_NN: Lower bounds of neural network output
        - UB_NN: Upper bounds of neural network output
    """
    
    # Sanity check
    assert LB_hidden_box_list is not None 
    assert UB_hidden_box_list is not None
    assert all( v <=1.0 or v>=0 for v in influence_threshold)

    # create manager for Elina
    man = elina_box_manager_alloc()
    # create gurobi model
    m = get_model()
    # create all gurobi variables for the network
    m, z, z_hat = add_all_vars(m, numlayer, LB_N0, UB_N0, UB_hidden_box_list)
    
    # initialize counter
    nn.ffn_counter = 0
    
    # Init statistics
    stats = {'time': [],
        'LB_hat': [],
        'UB_hat': [],
        'use_LP': [],
        'margin': [],
        'margin_per_neuron': [],
        'margin_per_time': [],
        'tightness_hat': [],
        'min_tightness_hat': [],
        'max_tightness_hat': [],
        'median_tightness_hat': [],
        }

    # Adding weights constraints for k layers
    for layerno in range(numlayer):
        if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
            t1 = time.time()
                        
            # read the layer weights and biases
            weights = nn.weights[nn.ffn_counter]
            biases = nn.biases[nn.ffn_counter]
            np.ascontiguousarray(weights, dtype=np.float)
            np.ascontiguousarray(biases, dtype=np.float)

            # output shape of the layer
            n_in = weights.shape[1]
            n_out = weights.shape[0]

            # create variables to store bounds of hidden layer
            LB_hat = np.zeros(n_out, float)
            UB_hat = np.zeros(n_out, float)

            # add affine constraint
            add_hidden_constraint(m, layerno, z[layerno], z_hat[layerno], weights, biases)
            
            ##########################################
            # First layer: Use original BOX bounds   #
            ##########################################
            if layerno < lp_list[0] or layerno == 0:
                use_LP = np.zeros(n_out, dtype=np.bool)
                LB_hat, UB_hat = LB_hidden_box_list[layerno].copy() , UB_hidden_box_list[layerno].copy()
            
            #########################################
            # Last Layer: Use Linear Programming    #
            #########################################
            elif layerno == numlayer - 1:
                use_LP = np.ones(n_out, dtype=np.bool)
                
                # LB: Get lower bound of correct label
                LB_hat[true_label], _ = call_linear_solver(m, z_hat[layerno][true_label], lb_only=True)
                
                # UB: Get upper bound of all other labels    
                for i_out in range(n_out):
                    if i_out == true_label:
                        continue
                        
                    _, UB_hat[i_out] = call_linear_solver(m, z_hat[layerno][i_out], ub_only=True)
                
            ######################################################
            # All layers inbetween: Decide if we use LP or BOX   #
            ######################################################
            else:
                
                # Calculate box to get tigthness
                LB, UB, n_out = get_relu_bounds_using_box(man, LB_hat_prev, UB_hat_prev, n_in)
                LB_hat, UB_hat, n_out = get_hidden_bounds_using_box(man, weights, biases, 
                                                                LB, UB, n_out, verbose)
                
#                 # Check if Relu Constrain will be non trivial
#                 trivial_relu_constrain = LB_hat > 0 or UB_hat < 0
#                 nontrivial_relu_constrain = not trivial_relu_constrain
                
                # Relu activate
                relu_inactive = UB_hat < 0
                relu_inactive = relu_inactive.squeeze()
            
                # Get Tighness Rank
                tightness_box = UB_hat.squeeze() - LB_hat.squeeze()
                tightness_box[relu_inactive] = 0
                temp = np.argsort(-tightness_box, axis=0)
                tightness_box_rank = np.empty_like(temp)
                tightness_box_rank[temp] = np.arange(len(temp))
                tightness_box_rank = tightness_box_rank/n_out
                
                # Get Influence Rank
                norm = 1
                w = nn.weights[layerno+1]
                norms = np.linalg.norm(w, ord=norm, axis=0)
                norms[relu_inactive] = 0
                influence = norms * tightness_box
                temp = np.argsort(-influence, axis=0)
                influence_rank = np.empty_like(temp)
                influence_rank[temp] = np.arange(len(temp))
                influence_rank = influence_rank/n_out
                
                # LP
                # If the layer is in the list of layer use LP on ALL neurons
                if (layerno in lp_list):
                    # Use LP if the influence_rank is below the specified threshold.
                    use_LP = (influence_rank < influence_threshold[0])
                    
                # LP
                # If the layer is NOT in list of layer use LP only on high ranking neurons
                else:
                    # Use LP if the influence_rank is below the specified threshold.
                    use_LP = (influence_rank < influence_threshold[1])
                
                for i_out in range(n_out):
                    if use_LP[i_out]:
                        LB_hat[i_out], UB_hat[i_out] = call_linear_solver(m, z_hat[layerno][i_out]) 
            
            # add relu constraint
            if layerno < (numlayer - 1) and nn.layertypes[layerno] in ["ReLU"]:
                add_relu_activation_constraint(m, layerno, z_hat[layerno], z[layerno + 1], LB_hat, UB_hat)
            
            # preparation for next iteration    
            LB_hat_prev, UB_hat_prev = LB_hat.copy(), UB_hat.copy()
                
            m.update()

            # update counter for next iteration
            nn.ffn_counter += 1
            # Save where we used LP
            nn.use_LP[layerno] = use_LP
            nn.LB_hat[layerno] = LB_hat
            nn.UB_hat[layerno] = UB_hat
            
            
            # Save stats
            t2 = time.time()
            
            stats['time'].append(t2 - t1)
            stats['LB_hat'].append(LB_hat.squeeze())
            stats['UB_hat'].append(UB_hat.squeeze())
            stats['use_LP'].append(use_LP)
            tightness = stats['UB_hat'][-1] - stats['LB_hat'][-1]
            stats['tightness_hat'].append(tightness)
            stats['median_tightness_hat'].append(np.median(tightness))
            stats['min_tightness_hat'].append(min(tightness))
            stats['max_tightness_hat'].append(max(tightness))

            if layerno == (numlayer - 1):
                stats['margin'].append(LB_hat[true_label] - max(UB_hat))
                n_use_lp = sum([sum(lp) for lp in stats['use_LP']])
                tot_time = sum(stats['time'])
                stats['margin_per_neuron'].append( stats['margin'][-1]/n_use_lp )
                stats['margin_per_time'].append(stats['margin'][-1]/tot_time)


            
            if verbose:
                decimals=2
                print("--------------------------")
                print("Layerno: %d" %layerno)
                print("Time %3f" % (t2 - t1))
                if stats['use_LP'][-1].any():
                    print("Time per LP neuron %3f" % ((t2 - t1)/ sum(stats['use_LP'][-1])))
                    print("LP used on %d neurons." % sum(stats['use_LP'][-1]))
                print("Median tigthness of hat bounds: %3f \n" % stats['median_tightness_hat'][-1])
                print("Min tigthness of hat bounds: %3f \n" % stats['min_tightness_hat'][-1])
                print("Max tigthness of hat bounds: %3f \n" % stats['max_tightness_hat'][-1])
                print("[LB_hat | UB_hat | use_LP]")
                pprint(np.stack([LB_hat.squeeze(), UB_hat.squeeze(), stats['use_LP'][-1]], axis = 1).round(decimals=decimals))
                print("--------------------------\n")

                if layerno == (numlayer - 1):
                    print("----------SUMMARY-----------\n")
                    print("Verification Margin (more positive better): %3f \n" % stats['margin'][-1])
                    print("Margin per LP neuron (more positve is better): %6f\n" % stats['margin_per_neuron'][-1])
                    print("Margin per second (more positve is better): %10f\n" % stats['margin_per_time'][-1])
                    print("----------END SUMMARY--------\n")

        else:
            raise("Not a valid layer!")
    
    # Set bounds of last performed layer to output
    LB_NN = LB_hat
    UB_NN = UB_hat
    # If last Layer is RELU change last lower and upper bounds accordingly.
    if nn.layertypes[-1] == "ReLU" :
        num_out = len(UB_hat)
        for i in range(num_out):
            if LB_hat[i] < 0 :
                LB_NN[i] = 0 
            if UB_hat[i] < 0 :
                UB_NN[i] = 0 
           
    return LB_NN, UB_NN, stats

# Initialize the problem variables

In [20]:
!ls /home/riai2018/mnist_nets/

mnist_relu_3_10.txt    mnist_relu_6_100.txt  mnist_relu_9_100.txt
mnist_relu_3_20.txt    mnist_relu_6_200.txt  mnist_relu_9_200.txt
mnist_relu_3_50.txt    mnist_relu_6_20.txt
mnist_relu_4_1024.txt  mnist_relu_6_50.txt


# Get perturbed label and Load NN (provided prediction for unperturbed is true)

In [21]:
def load_nn(netname, specname, epsilon):
    flag_wrong_label = False
    
    with open(netname, 'r') as netfile:
        netstring = netfile.read()
    with open(specname, 'r') as specfile:
        specstring = specfile.read()
    nn = parse_net(netstring)
    x0_low, x0_high = parse_spec(specstring)
    LB_N0, UB_N0 = get_perturbed_image(x0_low,0)
    
    label, _ = analyze(nn,LB_N0,UB_N0,0) # Get label of unperturbed image, i.e. eps=0
    
    print("True label: " + str(label))
    if(label == int(x0_low[0])):
        LB_N0, UB_N0 = get_perturbed_image(x0_low, epsilon)
    else:
        print("image not correctly classified by the network. expected label ",int(x0_low[0]), " classified label: ", label)
        flag_wrong_label =  True
        
    return LB_N0, UB_N0, nn, label, flag_wrong_label

In [22]:
netname = '/home/riai2018/mnist_nets/mnist_relu_6_20.txt'
specname = '/home/riai2018/mnist_images/img2.txt'
epsilon = 0.001
LB_N0, UB_N0, nn, label, flag_wrong_label = load_nn(netname, specname, epsilon)

True label: 1


In [23]:
numlayer = nn.numlayer 

for layerno in range(numlayer):
    if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
        print(nn.layertypes[layerno])

ReLU
ReLU
ReLU
ReLU
ReLU
ReLU


# Start Verification

## 1) Find naive/spectral bounds using box approximation

In [24]:
t1 = time.time()
LB_hidden_box_list, UB_hidden_box_list, LB_NN, UB_NN = perform_box_analysis(nn, LB_N0, UB_N0, verbose = False)
t2 = time.time()

In [25]:
# check verifiability with box!
print("--------- For Box ----------")
print("[OUTPUT] Bounds: ")
pprint(np.stack([LB_NN, UB_NN], axis = 1))
print("[VERIFY] Verification status: ")
t = time.time()
_, flag = verify_network(LB_N0, UB_N0, LB_NN, UB_NN, \
                         label, num_input_pixels = len(LB_N0), num_out_pixels = 10)
print("----------------------------")
print("Calculation time: %s s"% (t2-t1))

--------- For Box ----------
[OUTPUT] Bounds: 
array([[ 0.        ,  0.        ],
       [ 9.47404515, 10.8678384 ],
       [ 2.0732196 ,  2.86511957],
       [ 0.60576397,  1.06502746],
       [ 0.        ,  0.54891204],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 2.66937381,  3.50468927],
       [ 2.38386408,  3.36602735],
       [ 0.        ,  0.        ]])
[VERIFY] Verification status: 
verified
----------------------------
Calculation time: 0.12157368659973145 s


## 2) Find tighter bounds using linear programming over box intervals

__Paper:__ "Provable defenses against adversarial examples via the convex outer adversarial polytope"; J. Zico Kolter, Eric Wong [[PDF]](https://machine-learning-and-security.github.io/papers/mlsec17_paper_42.pdf)

We follow the following notation: 
```
Consider a k-layer feedforward ReLU-based NN, f_θ : R^|x| → R^|y|. Given equations: 
        z_hat_{i} = W_i * z_i + b_i, where i = {1, . . . , k}
        z_i = max(z_hat_{i-1} , 0),      where i = {2, . . . , k}
and, 
        z_1 = x
        f_θ(x) ≡ z_hat_{k}
```

In [26]:
t1 = time.time()
LB_NN, UB_NN = perform_linear_over_box_approximation(nn, LB_N0, UB_N0, LB_hidden_box_list, UB_hidden_box_list, verbose = False)
t2 = time.time()

Academic license - for non-commercial use only
Number of relu layers: 6
Number of hidden layers: 6
Size of last hidden layer: 10


In [27]:
# check verifiability with linear!
print("--------- For Linear (over box) ----------")
print("[OUTPUT] Bounds: ")
pprint(np.stack([LB_NN, UB_NN], axis = 1))
print("[VERIFY] Verification status: ")
_, flag = verify_network(LB_N0, UB_N0, LB_NN, UB_NN, \
                         label, num_input_pixels = len(LB_N0), num_out_pixels = 10)
print("----------------------------")
print("Calculation time: %4f s." % (t2-t1))

--------- For Linear (over box) ----------
[OUTPUT] Bounds: 
array([[ 0.        ,  0.        ],
       [10.0469925 , 10.33118855],
       [ 2.42420647,  2.4952212 ],
       [ 0.80795125,  0.84694697],
       [ 0.06546422,  0.13054258],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 2.9903704 ,  3.13904587],
       [ 2.84708954,  2.92978192],
       [ 0.        ,  0.        ]])
[VERIFY] Verification status: 
verified
----------------------------
Calculation time: 0.540243 s.


# 3) Optimize through linear only

In [28]:
# params
lp_freq = 1
lp_start = 1
verbose = True

lp_list = [i for i in range(lp_start, numlayer, lp_freq)]

In [29]:
t_1 =time.time()
LB_NN, UB_NN, stats = perform_linear_layerwise(nn, numlayer, LB_N0, UB_N0, lp_list,
                             LB_hidden_box_list, UB_hidden_box_list, label, verbose=verbose)
t_2 =time.time()
print(stats)

Number of relu layers: 6
Number of hidden layers: 6
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 0.080661
Median global rank: nan
Median tigthness of hat bounds: 0.084248 

Min tigthness of hat bounds: 0.067921 

Max tigthness of hat bounds: 0.109598 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-1.76, -1.67,   nan,   nan,  0.  ],
       [-1.98, -1.87,   nan,   nan,  0.  ],
       [-1.46, -1.39,   nan,   nan,  0.  ],
       [-0.07,  0.02,   nan,   nan,  0.  ],
       [-1.15, -1.05,   nan,   nan,  0.  ],
       [ 1.13,  1.22,   nan,   nan,  0.  ],
       [-0.73, -0.65,   nan,   nan,  0.  ],
       [ 0.51,  0.61,   nan,   nan,  0.  ],
       [-1.52, -1.43,   nan,   nan,  0.  ],
       [ 1.05,  1.14,   nan,   nan,  0.  ],
       [-3.48, -3.39,   nan,   nan,  0.  ],
       [-4.13, -4.06,   nan,   nan,  0.  ],
       [ 0.97,  1.06,   nan,   nan,  0.  ],
       [-0.24, -0.17,   nan,   nan,  0.  ],
       [ 1.11,  1.19,   nan,   nan,  0.  ],
     

/home/riai2018/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:3250: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


--------------------------
Layerno: 1
Time 0.455225
Time per LP neuron 0.022761
LP used on 20 neurons.
Median global rank: 0.181250
Median tigthness of hat bounds: 0.060159 

Min tigthness of hat bounds: 0.018668 

Max tigthness of hat bounds: 0.134819 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.6 , -0.57,  0.16,  0.36,  1.  ],
       [-0.02,  0.04,  0.02,  0.  ,  1.  ],
       [-0.49, -0.44,  0.34,  0.71,  1.  ],
       [-1.02, -0.96,  0.03,  0.06,  1.  ],
       [-0.83, -0.77,  0.17,  0.41,  1.  ],
       [ 5.69,  5.83,  0.33,  0.66,  1.  ],
       [ 1.65,  1.73,  0.59,  0.86,  1.  ],
       [-0.5 , -0.42,  0.07,  0.16,  1.  ],
       [-0.6 , -0.56,  0.06,  0.11,  1.  ],
       [-0.13, -0.08,  0.09,  0.21,  1.  ],
       [-0.03,  0.07,  0.61,  0.91,  1.  ],
       [-0.24, -0.2 ,  0.19,  0.5 ,  1.  ],
       [-0.66, -0.6 ,  0.18,  0.46,  1.  ],
       [ 1.13,  1.22,  0.49,  0.81,  1.  ],
       [ 4.91,  5.03,  0.24,  0.56,  1.  ],
       [-0.16, -0.08,  0.25,  0

In [30]:
print("------------TIME------------")
print("Total number of layers = %s"% numlayer)
print("lp_list = %s"% lp_list)
print("Time Solving LP: %4f s." % (t_2-t_1))
print("----------------------------")

# check verifiability with linear!
print("--------- For Linear ----------")
print("[OUTPUT] Bounds: ")
pprint(np.stack([LB_NN.squeeze(), UB_NN.squeeze()], axis = 1))
print("[VERIFY] Verification status: ")
_, flag = verify_network(LB_N0, UB_N0, LB_NN, UB_NN, \
                         label, num_input_pixels = len(LB_N0), num_out_pixels = 10)

------------TIME------------
Total number of layers = 6
lp_list = [1, 2, 3, 4, 5]
Time Solving LP: 1.692142 s.
----------------------------
--------- For Linear ----------
[OUTPUT] Bounds: 
array([[ 0.        ,  0.        ],
       [10.09297108,  0.        ],
       [ 0.        ,  2.48018364],
       [ 0.        ,  0.8327827 ],
       [ 0.        ,  0.12350749],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  3.08866122],
       [ 0.        ,  2.92154509],
       [ 0.        ,  0.        ]])
[VERIFY] Verification status: 
verified


# 4) Speed Linear Solver up

In [31]:
# params
lp_freq = 2
lp_start = 1
verbose = True

lp_list = [i for i in range(lp_start, numlayer, lp_freq)]
lp_list = [i for i in range(lp_start, numlayer-2)]
print(lp_list)

[1, 2, 3]


In [32]:
t_1 =time.time()
LB_NN, UB_NN, stats = perform_linear_layerwise(nn, numlayer, LB_N0, UB_N0, lp_list,
                             LB_hidden_box_list, UB_hidden_box_list, label, verbose=verbose, global_rank_threshold=0.0,
                                       local_rank_threshold=0.0)
t_2 =time.time()

TypeError: object of type 'float' has no len()

In [ ]:
print(stats)

In [ ]:
print("------------TIME------------")
print("Total number of layers = %s"% numlayer)
print("lp_list = %s"% lp_list)
print("Time Solving LP: %4f s." % (t_2-t_1))
print("----------------------------")

# check verifiability with linear!
print("--------- For Linear ----------")
print("[OUTPUT] Bounds: ")
pprint(np.stack([LB_NN.squeeze(), UB_NN.squeeze()], axis = 1))
print("[VERIFY] Verification status: ")
_, flag = verify_network(LB_N0, UB_N0, LB_NN, UB_NN, \
                         label, num_input_pixels = len(LB_N0), num_out_pixels = 10)

# 5) Testing Script

In [47]:
def analyse_nn_and_write(net_code, img_nrs, epsilon, log_file=None, verbose=True, **kwargs):
    """
    Analyse Neural Network and write the result to a log file. Code heuristic yourself in this function
    INPUT:
        - net_code: Neural Network code, e.g. mnist_relu_4_1024
        - img_nrs: List of image nrs, e.g [0,1,2,3]
        - epsilon: Perturbation
        - log_file: File to write results
    OUTPUT:
        - verified_flag: Verification flag. True if network was verified against perturbation.
    """
    nn_root_path = '/home/riai2018/mnist_nets'
    img_root_path = '/home/riai2018/mnist_images'
    log_root_path = '/home/riai2018/log'
    log_base_name = 'log_'
    verified_counter, true_label_counter = 0, 0
    stamp = '_{:%Y-%m-%d_%H:%M:%S}'.format(datetime.now())
    if not os.path.isdir(log_root_path):
        os.makedirs(log_root_path)
    net_path = os.path.join(nn_root_path, net_code + '.txt')
    if log_file is None:
        log_file = os.path.join(log_root_path, log_base_name + net_code + '_epsilon_' + str(epsilon))
        if kwargs is not None:
            log_file = log_file + '_'  + str(kwargs) + "_"
        log_file = log_file + stamp +'.txt'
            
    with open(log_file, "+a") as f:
        f.write("Epsilon %s kwargs: %s \n\n" % (epsilon, kwargs))
        
    assert max(img_nrs) <= 99
    assert min(img_nrs) >= 0
    t_start = time.time()
    
    runtimes = []
    
    for img_nr in img_nrs:
        img_path = os.path.join(img_root_path, 'img' + str(img_nr) + '.txt')

        # Load NN and perturbe image
        LB_N0, UB_N0, nn, label, flag_wrong_label = load_nn(net_path, img_path, epsilon)
        true_label_counter += int(not flag_wrong_label)
        numlayer = nn.numlayer

        # You heuristics come here:
#         lp_freq = 2
        lp_start = 1
        numlayer = nn.numlayer
#         lp_list = [i for i in range(lp_start, numlayer, lp_freq)]
        lp_list = [i for i in range(lp_start, numlayer-2)]

        # Get Bounds
        t1 =time.time()
        LB_hidden_box_list, UB_hidden_box_list, LB_NN, UB_NN = perform_box_analysis(nn, LB_N0, UB_N0, verbose = False)
        LB_NN, UB_NN, stats = perform_linear_layerwise(nn, numlayer, LB_N0, UB_N0, lp_list,
                                     LB_hidden_box_list, UB_hidden_box_list, label, verbose=verbose, **kwargs)
        # Check if NN was verified
        _, verified_flag = verify_network(LB_N0, UB_N0, LB_NN, UB_NN, label, num_input_pixels = len(LB_N0), num_out_pixels = 10)
        verified_counter += int(verified_flag)
       
        t2 = time.time()
        
        runtimes.append(t2-t1)
        
        # Write to logfile
        with open(log_file, "+a") as f:
            if not flag_wrong_label and verified_flag:
                line = "{:>10} img_{:>2} verified label {:>2} time {:>5.4f} s\n\n".format(net_code, img_nr, label, t2-t1)
            elif not flag_wrong_label and not verified_flag:
                line = "{:>10} img_{:>2} failed label {:>2} time {:>5.4f} s\n\n".format(net_code, img_nr, label, t2-t1)
            else:
                line = "{:>10} img_{:>2} not considered\n\n".format(net_code, img_nr)
            f.write(line)
            f.write("Layerwise_time: %s \n" % stats['time'])
            bounds = np.stack([stats['LB_hat'][-1], stats['UB_hat'][-1]], axis = 1) 
            f.write("Final Bounds: %s \n" % (bounds))
            f.write("Final Verification Margin (negative is not verified): %3f \n" % stats['margin'][-1])
            f.write("Margin per Neuron (more positive is better): %9f \n" % stats['margin_per_neuron'][-1])
            f.write("Margin per Second (more positve is better): %10f\n" % stats['margin_per_time'][-1])
            f.write("\n-------------------------------------------------------------------------------------\n\n")

            
    with open(log_file, "+a") as f:
        line = []
        line.append("\n--------------------- \n")
        line.append("analysis precision  {:>2} /  {:>2}\n".format(verified_counter, true_label_counter))
        line.append("Average Time: {:>5.4f} \n".format((time.time() - t_start)/len(img_nrs)))
        line.append("Max Time: {:>5.4f} \n".format(max(runtimes)))
        line.append("Min Time: {:>5.4f} \n".format(min(runtimes)))
        for l in line:
            f.write(l)

    return stats

In [34]:
# Create list of all NN and images

directory = '../mnist_nets/'
nn_list  = os.listdir(directory)
nn_list = [f.replace('.txt','') for f in nn_list]
nn_list.sort()
print("All Neural Networks: %s" % nn_list)
directory = '../mnist_images/'
img_nrs  = np.arange(100).tolist()
verbose = False
# Create list of all groundtruth NN and epsilons.

All Neural Networks: ['mnist_relu_3_10', 'mnist_relu_3_20', 'mnist_relu_3_50', 'mnist_relu_4_1024', 'mnist_relu_6_100', 'mnist_relu_6_20', 'mnist_relu_6_200', 'mnist_relu_6_50', 'mnist_relu_9_100', 'mnist_relu_9_200']


In [44]:
verbose=True
analyse_nn_and_write('mnist_relu_6_20', [2, 10] , 0.001, verbose=verbose, global_rank_threshold=[0.0, 0.0],
                    local_rank_threshold=[0.0,0.0], tightness_threshold=[0.0, 0.0], influence_threshold=[0.3, 0.1])

True label: 1
Number of relu layers: 6
Number of hidden layers: 6
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 0.032199
Median global rank: nan
Median tigthness of hat bounds: 0.084248 

Min tigthness of hat bounds: 0.067921 

Max tigthness of hat bounds: 0.109598 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-1.76, -1.67,   nan,   nan,  0.  ],
       [-1.98, -1.87,   nan,   nan,  0.  ],
       [-1.46, -1.39,   nan,   nan,  0.  ],
       [-0.07,  0.02,   nan,   nan,  0.  ],
       [-1.15, -1.05,   nan,   nan,  0.  ],
       [ 1.13,  1.22,   nan,   nan,  0.  ],
       [-0.73, -0.65,   nan,   nan,  0.  ],
       [ 0.51,  0.61,   nan,   nan,  0.  ],
       [-1.52, -1.43,   nan,   nan,  0.  ],
       [ 1.05,  1.14,   nan,   nan,  0.  ],
       [-3.48, -3.39,   nan,   nan,  0.  ],
       [-4.13, -4.06,   nan,   nan,  0.  ],
       [ 0.97,  1.06,   nan,   nan,  0.  ],
       [-0.24, -0.17,   nan,   nan,  0.  ],
       [ 1.11,  1.19,   nan,   nan,

Number of relu layers: 6
Number of hidden layers: 6
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 0.042750
Median global rank: nan
Median tigthness of hat bounds: 0.094487 

Min tigthness of hat bounds: 0.075145 

Max tigthness of hat bounds: 0.117509 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[ -1.26,  -1.17,    nan,    nan,   0.  ],
       [  7.8 ,   7.92,    nan,    nan,   0.  ],
       [  6.54,   6.62,    nan,    nan,   0.  ],
       [ -4.7 ,  -4.6 ,    nan,    nan,   0.  ],
       [ -1.84,  -1.73,    nan,    nan,   0.  ],
       [ -6.58,  -6.47,    nan,    nan,   0.  ],
       [ -7.99,  -7.9 ,    nan,    nan,   0.  ],
       [ -2.71,  -2.6 ,    nan,    nan,   0.  ],
       [  1.41,   1.51,    nan,    nan,   0.  ],
       [  8.43,   8.53,    nan,    nan,   0.  ],
       [ -3.16,  -3.07,    nan,    nan,   0.  ],
       [  4.95,   5.04,    nan,    nan,   0.  ],
       [ -3.13,  -3.04,    nan,    nan,   0.  ],
       [ -5.51,  -5.43,    n

{'time': [0.042749643325805664,
  0.06226205825805664,
  0.06330156326293945,
  0.16518068313598633,
  0.029363632202148438,
  0.05474138259887695],
 'LB_hat': [array([ -1.26549334,   7.79631979,   6.53588223,  -4.70052856,
          -1.84236632,  -6.58231054,  -7.99787284,  -2.71496696,
           1.40617831,   8.4212984 ,  -3.16152067,   4.94829639,
          -3.13571021,  -5.51140898,  -3.83282426,  -0.88180955,
           2.07318787, -12.64210322,  -0.36338058,   0.22995437]),
  array([-1.42051807, -5.42877403, -1.0961845 ,  6.2877356 ,  8.9673463 ,
         -3.9073478 , -2.87461919, -1.78269081,  4.16404499, -3.71106275,
         -5.28039444, -2.01104371, -1.70291204, -0.40627105, -4.05005136,
         -3.83682073, -0.85056194, -0.04025076, -5.16081344, -3.56168882]),
  array([-0.73816649, -4.52844373,  4.36188344, -1.794051  ,  0.0954157 ,
         -2.42610735,  1.25077603, -2.68432905, -3.21925747,  5.47647721,
         -2.4290538 ,  0.19288418, -3.29416781,  0.77548775, -0.7525

# Test Ranking

In [37]:
verbose=True
imgs = [10]

# New Ranking function axis=0 But still wrong

In [ ]:
stats = analyse_nn_and_write('mnist_relu_4_1024', imgs , 0.001, global_rank_threshold=[0.0,0.0], local_rank_threshold=[0.3, 0.1])

# New Corrected Ranking Function

In [38]:
stats = analyse_nn_and_write('mnist_relu_4_1024', imgs , 0.001, global_rank_threshold=[0.0,0.0], 
                             local_rank_threshold=[0.3, 0.1])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.694447
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 338.267094
Time per LP neuron 1.098270


# New Corrected Ranking function with TIIIGTHNESS

In [40]:
stats = analyse_nn_and_write('mnist_relu_4_1024', imgs , 0.001, global_rank_threshold=[0.0,0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.3,0.1])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.745458
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 405.985557
Time per LP neuron 1.318135


# New Corrected Ranking function with Influence

In [56]:
stats = analyse_nn_and_write('mnist_relu_4_1024', imgs , 0.001, global_rank_threshold=[0.0,0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0,0.0],
                            influence_threshold=[0.3, 0.1])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.767830
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 335.412481
Time per LP neuron 1.089002


In [38]:
stats = analyse_nn_and_write('mnist_relu_4_1024', imgs , 0.001, global_rank_threshold=[0.0,0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0,0.0],
                            influence_threshold=[0.1, 0.3])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 2.580826
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 122.311486
Time per LP neuron 1.187490


In [39]:
stats = analyse_nn_and_write('mnist_relu_4_1024', imgs , 0.001, global_rank_threshold=[0.0,0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0,0.0],
                            influence_threshold=[0.4, 0.0])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.741887
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 424.538470
Time per LP neuron 1.035460


In [40]:
stats = analyse_nn_and_write('mnist_relu_4_1024', imgs , 0.001, global_rank_threshold=[0.0,0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0,0.0],
                            influence_threshold=[0.5, 0.0])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.828523
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 495.623662
Time per LP neuron 0.968015


In [41]:
stats = analyse_nn_and_write('mnist_relu_4_1024', imgs , 0.001, global_rank_threshold=[0.0,0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0,0.0],
                            influence_threshold=[0.6, 0.0])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.761425
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 586.068258
Time per LP neuron 0.952957


In [43]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10,2] , 0.001, global_rank_threshold=[0.0,0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0,0.0],
                            influence_threshold=[0.7, 0.0])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 2.341876
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 930.011757
Time per LP neuron 1.297088


GurobiError: Out of memory

In [42]:
stats = analyse_nn_and_write('mnist_relu_4_1024', imgs , 0.001, global_rank_threshold=[0.001,0.01], 
                             local_rank_threshold=[0.001, 0.01], tightness_threshold=[0.001,0.01],
                            influence_threshold=[0.3, 0.1])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.721893
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 295.680515
Time per LP neuron 0.960002


# Again Fixed weights and added relu inactive

In [38]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.1, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.1, 0.0],
                            influence_threshold=[0.3, 0.1])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 2.014191
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 430.476153
Time per LP neuron 1.024943


In [39]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.1, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.1, 0.0],
                            influence_threshold=[0.4, 0.0])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.729687
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 530.498190
Time per LP neuron 1.048415


In [40]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.1, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.1, 0.0],
                            influence_threshold=[0.5, 0.0])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.749041
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 575.825227
Time per LP neuron 0.972678


In [41]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.1, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.1, 0.0],
                            influence_threshold=[0.6, 0.0])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.747272
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 652.014636
Time per LP neuron 0.960257


# Changed weights back to old definition

In [40]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.1, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.1, 0.0],
                            influence_threshold=[0.3, 0.1])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.866492
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 366.662338
Time per LP neuron 0.918953


# Changed weights to layerno and fixed relu inactive constrain

In [48]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.1, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.1, 0.0],
                            influence_threshold=[0.3, 0.0])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.906172
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 384.298464
Time per LP neuron 0.937313


In [36]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.0, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0, 0.0],
                            influence_threshold=[0.3, 0.0])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 2.676781
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 300.788319
Time per LP neuron 0.976585


In [37]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.0, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0, 0.0],
                            influence_threshold=[0.3, 0.1])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.656990
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 289.094816
Time per LP neuron 0.938620


In [38]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.0, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0, 0.0],
                            influence_threshold=[0.4, 0.])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.649722
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 374.888990
Time per LP neuron 0.914363


In [39]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.0, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.4, 0.0],
                            influence_threshold=[0.0, 0.0])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.663882
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 371.824993
Time per LP neuron 0.906890


In [40]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.0, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0, 0.0],
                            influence_threshold=[0.1, 0.3])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.634518
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 99.113219
Time per LP neuron 0.962264
L

In [42]:
stats = analyse_nn_and_write('mnist_relu_4_1024', [10] , 0.001, global_rank_threshold=[0.0, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0, 0.0],
                            influence_threshold=[0.35, 0.0])

True label: 0
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.668581
Median global rank: nan
Median tigthness of hat bounds: 0.030266 

Min tigthness of hat bounds: 0.025867 

Max tigthness of hat bounds: 0.040032 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.55, -0.52,   nan,   nan,  0.  ],
       [-0.02,  0.01,   nan,   nan,  0.  ],
       [-0.51, -0.48,   nan,   nan,  0.  ],
       ...,
       [ 1.49,  1.52,   nan,   nan,  0.  ],
       [-0.76, -0.73,   nan,   nan,  0.  ],
       [ 0.7 ,  0.73,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 388.825625
Time per LP neuron 1.083080


In [36]:
stats = analyse_nn_and_write('mnist_relu_4_1024', img_nrs , 0.001, global_rank_threshold=[0.0, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0, 0.0],
                            influence_threshold=[0.4, 0.0])

True label: 7
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.974612
Median global rank: nan
Median tigthness of hat bounds: 0.028662 

Min tigthness of hat bounds: 0.024511 

Max tigthness of hat bounds: 0.037791 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[ 0.12,  0.15,   nan,   nan,  0.  ],
       [-0.15, -0.13,   nan,   nan,  0.  ],
       [-0.72, -0.69,   nan,   nan,  0.  ],
       ...,
       [-0.79, -0.76,   nan,   nan,  0.  ],
       [-0.8 , -0.77,   nan,   nan,  0.  ],
       [-0.28, -0.26,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 451.199348
Time per LP neuron 1.100486


[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 2
Time 7.442212
Median global rank: 0.556641
Median tigthness of hat bounds: 1.657892 

Min tigthness of hat bounds: 1.437870 

Max tigthness of hat bounds: 1.885328 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.6 ,  0.93,  0.35,  0.27,  0.  ],
       [-1.  ,  0.6 ,  0.79,  0.76,  0.  ],
       [ 0.6 ,  2.2 ,  0.49,  0.41,  0.  ],
       ...,
       [-0.7 ,  0.85,  0.4 ,  0.31,  0.  ],
       [-0.44,  1.19,  0.22,  0.15,  0.  ],
       [-0.74,  0.9 ,  0.25,  0.16,  0.  ]])
--------------------------

--------------------------
Layerno: 3
Time 43.445123
Time per LP neuron 4.344512
LP used on 10 neurons.
Median global rank: nan
Median tigthness of hat bounds: 3.835737 

Mi

True label: 1
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.763222
Median global rank: nan
Median tigthness of hat bounds: 0.027634 

Min tigthness of hat bounds: 0.023654 

Max tigthness of hat bounds: 0.036135 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.96, -0.93,   nan,   nan,  0.  ],
       [-0.09, -0.07,   nan,   nan,  0.  ],
       [ 0.02,  0.05,   nan,   nan,  0.  ],
       ...,
       [ 0.6 ,  0.63,   nan,   nan,  0.  ],
       [ 0.34,  0.37,   nan,   nan,  0.  ],
       [-0.15, -0.12,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 405.398152
Time per LP neuron 0.988776


[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 2
Time 7.073435
Median global rank: 0.556641
Median tigthness of hat bounds: 1.639541 

Min tigthness of hat bounds: 1.416202 

Max tigthness of hat bounds: 1.875456 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.99,  0.63,  0.35,  0.27,  0.  ],
       [-1.53,  0.11,  0.79,  0.76,  0.  ],
       [-0.49,  1.21,  0.49,  0.41,  0.  ],
       ...,
       [-1.07,  0.56,  0.4 ,  0.31,  0.  ],
       [-0.96,  0.73,  0.22,  0.15,  0.  ],
       [-0.13,  1.47,  0.25,  0.16,  0.  ]])
--------------------------



GurobiError: Out of memory

In [ ]:
stats = analyse_nn_and_write('mnist_relu_4_1024', img_nrs , 0.001, global_rank_threshold=[0.0, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0, 0.0],
                            influence_threshold=[0.3, 0.0])

True label: 7
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.685140
Median global rank: nan
Median tigthness of hat bounds: 0.028662 

Min tigthness of hat bounds: 0.024511 

Max tigthness of hat bounds: 0.037791 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[ 0.12,  0.15,   nan,   nan,  0.  ],
       [-0.15, -0.13,   nan,   nan,  0.  ],
       [-0.72, -0.69,   nan,   nan,  0.  ],
       ...,
       [-0.79, -0.76,   nan,   nan,  0.  ],
       [-0.8 , -0.77,   nan,   nan,  0.  ],
       [-0.28, -0.26,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 309.426209
Time per LP neuron 1.004631


[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 2
Time 6.961875
Median global rank: 0.556641
Median tigthness of hat bounds: 2.378926 

Min tigthness of hat bounds: 2.104668 

Max tigthness of hat bounds: 2.680132 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.99,  1.31,  0.35,  0.27,  0.  ],
       [-1.31,  0.98,  0.79,  0.76,  0.  ],
       [ 0.29,  2.57,  0.49,  0.41,  0.  ],
       ...,
       [-1.02,  1.21,  0.4 ,  0.31,  0.  ],
       [-0.77,  1.53,  0.22,  0.15,  0.  ],
       [-1.14,  1.25,  0.25,  0.16,  0.  ]])
--------------------------

--------------------------
Layerno: 3
Time 41.267953
Time per LP neuron 4.126795
LP used on 10 neurons.
Median global rank: nan
Median tigthness of hat bounds: 10.511221 

M

True label: 1
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.632065
Median global rank: nan
Median tigthness of hat bounds: 0.027634 

Min tigthness of hat bounds: 0.023654 

Max tigthness of hat bounds: 0.036135 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.96, -0.93,   nan,   nan,  0.  ],
       [-0.09, -0.07,   nan,   nan,  0.  ],
       [ 0.02,  0.05,   nan,   nan,  0.  ],
       ...,
       [ 0.6 ,  0.63,   nan,   nan,  0.  ],
       [ 0.34,  0.37,   nan,   nan,  0.  ],
       [-0.15, -0.12,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 294.578977
Time per LP neuron 0.956425


In [ ]:
stats = analyse_nn_and_write('mnist_relu_4_1024', img_nrs , 0.001, global_rank_threshold=[0.0, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0, 0.0],
                            influence_threshold=[0.5, 0.0])

True label: 7
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 2.040956
Median global rank: nan
Median tigthness of hat bounds: 0.028662 

Min tigthness of hat bounds: 0.024511 

Max tigthness of hat bounds: 0.037791 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[ 0.12,  0.15,   nan,   nan,  0.  ],
       [-0.15, -0.13,   nan,   nan,  0.  ],
       [-0.72, -0.69,   nan,   nan,  0.  ],
       ...,
       [-0.79, -0.76,   nan,   nan,  0.  ],
       [-0.8 , -0.77,   nan,   nan,  0.  ],
       [-0.28, -0.26,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 526.511029
Time per LP neuron 1.028342


[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 2
Time 6.605385
Median global rank: 0.556641
Median tigthness of hat bounds: 0.959651 

Min tigthness of hat bounds: 0.808833 

Max tigthness of hat bounds: 1.098603 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.29,  0.62,  0.35,  0.27,  0.  ],
       [-0.66,  0.28,  0.79,  0.76,  0.  ],
       [ 0.93,  1.89,  0.49,  0.41,  0.  ],
       ...,
       [-0.39,  0.54,  0.4 ,  0.31,  0.  ],
       [-0.1 ,  0.85,  0.22,  0.15,  0.  ],
       [-0.41,  0.54,  0.25,  0.16,  0.  ]])
--------------------------

--------------------------
Layerno: 3
Time 38.973662
Time per LP neuron 3.897366
LP used on 10 neurons.
Median global rank: nan
Median tigthness of hat bounds: -1.365687 

M

True label: 1
Number of relu layers: 4
Number of hidden layers: 4
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 1.655506
Median global rank: nan
Median tigthness of hat bounds: 0.027634 

Min tigthness of hat bounds: 0.023654 

Max tigthness of hat bounds: 0.036135 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-0.96, -0.93,   nan,   nan,  0.  ],
       [-0.09, -0.07,   nan,   nan,  0.  ],
       [ 0.02,  0.05,   nan,   nan,  0.  ],
       ...,
       [ 0.6 ,  0.63,   nan,   nan,  0.  ],
       [ 0.34,  0.37,   nan,   nan,  0.  ],
       [-0.15, -0.12,   nan,   nan,  0.  ]])
--------------------------

[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
[Network] Input pixels: 1024
[Network] Shape of weights: (1024, 1024)
[Network] Shape of biases: (1024,)
[Network] Out pixels: 1024
--------------------------
Layerno: 1
Time 476.250762
Time per LP neuron 0.930177


# Other Networks

In [36]:
nn_list

['mnist_relu_3_10',
 'mnist_relu_3_20',
 'mnist_relu_3_50',
 'mnist_relu_4_1024',
 'mnist_relu_6_100',
 'mnist_relu_6_20',
 'mnist_relu_6_200',
 'mnist_relu_6_50',
 'mnist_relu_9_100',
 'mnist_relu_9_200']

In [44]:
stats = analyse_nn_and_write('mnist_relu_9_100', [10] , 0.1, global_rank_threshold=[0.0, 0.0], 
                             local_rank_threshold=[0.0, 0.0], tightness_threshold=[0.0, 0.0],
                            influence_threshold=[0.5, 0.0])

True label: 0
Number of relu layers: 9
Number of hidden layers: 9
Size of last hidden layer: 10
--------------------------
Layerno: 0
Time 0.405502
Median global rank: nan
Median tigthness of hat bounds: 6.661518 

Min tigthness of hat bounds: 6.119983 

Max tigthness of hat bounds: 8.992730 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[ -6.4 ,   0.78,    nan,    nan,   0.  ],
       [ -4.22,   2.34,    nan,    nan,   0.  ],
       [ -7.77,  -0.83,    nan,    nan,   0.  ],
       [ -8.35,  -1.03,    nan,    nan,   0.  ],
       [ -4.69,   2.25,    nan,    nan,   0.  ],
       [ -4.64,   1.58,    nan,    nan,   0.  ],
       [ -4.72,   1.96,    nan,    nan,   0.  ],
       [ -1.81,   6.02,    nan,    nan,   0.  ],
       [ -5.03,   1.42,    nan,    nan,   0.  ],
       [ -3.36,   3.04,    nan,    nan,   0.  ],
       [-10.01,  -2.53,    nan,    nan,   0.  ],
       [ -4.59,   2.03,    nan,    nan,   0.  ],
       [ -2.45,   5.26,    nan,    nan,   0.  ],
       [-12.09

--------------------------
Layerno: 1
Time 20.396812
Time per LP neuron 0.203968
LP used on 100 neurons.
Median global rank: 0.497857
Median tigthness of hat bounds: 22.556801 

Min tigthness of hat bounds: 14.557036 

Max tigthness of hat bounds: 33.312678 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-1.073e+01,  6.270e+00,  1.000e-01,  1.600e-01,  1.000e+00],
       [-1.039e+01,  6.080e+00,  4.900e-01,  4.800e-01,  1.000e+00],
       [-1.645e+01,  1.179e+01,  2.700e-01,  3.000e-01,  0.000e+00],
       [-2.056e+01,  1.146e+01,  1.200e-01,  1.900e-01,  0.000e+00],
       [-9.210e+00,  6.510e+00,  7.300e-01,  7.500e-01,  1.000e+00],
       [-1.927e+01,  9.150e+00,  5.700e-01,  5.600e-01,  0.000e+00],
       [-1.606e+01,  1.440e+01,  3.600e-01,  3.800e-01,  0.000e+00],
       [-1.787e+01,  1.087e+01,  1.000e-01,  1.600e-01,  0.000e+00],
       [-1.217e+01,  4.430e+00,  6.300e-01,  6.000e-01,  1.000e+00],
       [-8.390e+00,  7.070e+00,  5.300e-01,  5.200e-01,  1.000e+0

--------------------------
Layerno: 2
Time 28.308021
Time per LP neuron 0.283080
LP used on 100 neurons.
Median global rank: 0.476071
Median tigthness of hat bounds: 110.117277 

Min tigthness of hat bounds: 78.079737 

Max tigthness of hat bounds: 155.293516 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-4.4240e+01,  4.0630e+01,  2.6000e-01,  2.1000e-01,  1.0000e+00],
       [-4.9420e+01,  3.5840e+01,  4.7000e-01,  4.9000e-01,  1.0000e+00],
       [-4.9080e+01,  2.9240e+01,  4.2000e-01,  4.0000e-01,  1.0000e+00],
       [-4.1780e+01,  5.1960e+01,  7.8000e-01,  8.0000e-01,  1.0000e+00],
       [-4.9760e+01,  4.3210e+01,  2.4000e-01,  1.8000e-01,  1.0000e+00],
       [-7.9400e+01,  5.1900e+01,  8.2000e-01,  8.3000e-01,  0.0000e+00],
       [-8.9460e+01,  4.3410e+01,  7.6000e-01,  7.7000e-01,  0.0000e+00],
       [-8.3210e+01,  5.6890e+01,  9.5000e-01,  9.7000e-01,  0.0000e+00],
       [-4.7260e+01,  4.2830e+01,  6.0000e-02,  4.0000e-02,  1.0000e+00],
       [-7.5770e+0

--------------------------
Layerno: 3
Time 67.903631
Time per LP neuron 0.679036
LP used on 100 neurons.
Median global rank: 0.539286
Median tigthness of hat bounds: 565.371626 

Min tigthness of hat bounds: 385.266800 

Max tigthness of hat bounds: 781.524809 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-2.5688e+02,  1.2838e+02,  4.2000e-01,  4.0000e-01,  1.0000e+00],
       [-3.4398e+02,  3.2237e+02,  4.4000e-01,  4.3000e-01,  0.0000e+00],
       [-3.6984e+02,  2.9524e+02,  2.5000e-01,  2.2000e-01,  0.0000e+00],
       [-2.3491e+02,  1.6360e+02,  2.4000e-01,  2.1000e-01,  1.0000e+00],
       [-3.5262e+02,  3.2339e+02,  5.0000e-02,  3.0000e-02,  0.0000e+00],
       [-2.8880e+02,  1.7002e+02,  5.4000e-01,  5.0000e-01,  1.0000e+00],
       [-4.1381e+02,  2.9034e+02,  7.1000e-01,  7.1000e-01,  0.0000e+00],
       [-3.7571e+02,  3.2791e+02,  5.5000e-01,  5.0000e-01,  0.0000e+00],
       [-3.6396e+02,  3.0040e+02,  1.5000e-01,  1.2000e-01,  0.0000e+00],
       [-3.4661e+

--------------------------
Layerno: 4
Time 123.478542
Time per LP neuron 1.234785
LP used on 100 neurons.
Median global rank: 0.559643
Median tigthness of hat bounds: 2917.434404 

Min tigthness of hat bounds: 1945.853207 

Max tigthness of hat bounds: 3881.432320 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-1.80040e+03,  1.67374e+03,  2.00000e-01,  1.80000e-01,
         0.00000e+00],
       [-1.34836e+03,  7.29860e+02,  7.10000e-01,  7.00000e-01,
         1.00000e+00],
       [-1.19421e+03,  8.82950e+02,  6.30000e-01,  6.00000e-01,
         1.00000e+00],
       [-1.59802e+03,  6.96020e+02,  2.90000e-01,  2.50000e-01,
         1.00000e+00],
       [-1.84804e+03,  1.60536e+03,  5.00000e-01,  4.40000e-01,
         0.00000e+00],
       [-1.42202e+03,  8.26330e+02,  8.00000e-01,  7.80000e-01,
         1.00000e+00],
       [-1.90970e+03,  1.53580e+03,  8.70000e-01,  8.70000e-01,
         0.00000e+00],
       [-2.14473e+03,  1.27170e+03,  6.80000e-01,  6.60000e-01,
      

--------------------------
Layerno: 5
Time 186.795487
Time per LP neuron 1.867955
LP used on 100 neurons.
Median global rank: 0.481071
Median tigthness of hat bounds: 15010.260945 

Min tigthness of hat bounds: 9558.808324 

Max tigthness of hat bounds: 19438.505239 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-5.793670e+03,  4.611990e+03,  5.000000e-01,  5.200000e-01,
         1.000000e+00],
       [-9.799830e+03,  7.506880e+03,  7.000000e-02,  7.000000e-02,
         0.000000e+00],
       [-1.074916e+04,  7.651350e+03,  1.400000e-01,  1.300000e-01,
         0.000000e+00],
       [-7.060760e+03,  4.277710e+03,  5.900000e-01,  6.400000e-01,
         1.000000e+00],
       [-8.654170e+03,  7.161460e+03,  5.300000e-01,  5.600000e-01,
         0.000000e+00],
       [-6.286640e+03,  4.587980e+03,  4.700000e-01,  4.700000e-01,
         1.000000e+00],
       [-1.057765e+04,  6.621100e+03,  6.300000e-01,  6.600000e-01,
         0.000000e+00],
       [-1.025652e+04,  6.763840e

--------------------------
Layerno: 6
Time 286.558246
Time per LP neuron 2.865582
LP used on 100 neurons.
Median global rank: 0.538214
Median tigthness of hat bounds: 77724.690745 

Min tigthness of hat bounds: 47594.117378 

Max tigthness of hat bounds: 98288.923369 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-4.342127e+04,  3.653225e+04,  5.000000e-02,  7.000000e-02,
         0.000000e+00],
       [-3.395565e+04,  1.958399e+04,  4.100000e-01,  4.000000e-01,
         1.000000e+00],
       [-3.576158e+04,  2.138105e+04,  5.300000e-01,  5.000000e-01,
         1.000000e+00],
       [-2.781152e+04,  2.830942e+04,  5.400000e-01,  5.000000e-01,
         1.000000e+00],
       [-3.968655e+04,  4.975539e+04,  9.600000e-01,  9.500000e-01,
         0.000000e+00],
       [-5.117071e+04,  3.062729e+04,  3.800000e-01,  3.800000e-01,
         0.000000e+00],
       [-3.712630e+04,  1.739246e+04,  8.900000e-01,  8.600000e-01,
         1.000000e+00],
       [-2.786739e+04,  2.620744

[Network] Input pixels: 200
[Network] Shape of weights: (200, 200)
[Network] Shape of biases: (200,)
[Network] Out pixels: 200
--------------------------
Layerno: 7
Time 0.353429
Median global rank: 0.413929
Median tigthness of hat bounds: 463413.247858 

Min tigthness of hat bounds: 382541.422341 

Max tigthness of hat bounds: 594420.397433 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-2.6219554e+05,  1.7655115e+05,  5.4000000e-01,  6.1000000e-01,
         0.0000000e+00],
       [-2.4391742e+05,  2.3427647e+05,  2.5000000e-01,  2.9000000e-01,
         0.0000000e+00],
       [-3.0887987e+05,  1.5105707e+05,  4.4000000e-01,  5.3000000e-01,
         0.0000000e+00],
       [-3.5257208e+05,  1.9102971e+05,  3.4000000e-01,  3.7000000e-01,
         0.0000000e+00],
       [-2.6012834e+05,  2.2459617e+05,  6.3000000e-01,  7.0000000e-01,
         0.0000000e+00],
       [-3.0375149e+05,  1.8585105e+05,  7.4000000e-01,  7.5000000e-01,
         0.0000000e+00],
       [-3.0760474

--------------------------
Layerno: 8
Time 18.958779
Time per LP neuron 1.895878
LP used on 10 neurons.
Median global rank: nan
Median tigthness of hat bounds: 1653373.436665 

Min tigthness of hat bounds: 1211830.530964 

Max tigthness of hat bounds: 5122620.004829 

[LB_hat | UB_hat | global_rank | local_rank | use_LP]
array([[-5.12262000e+06,  0.00000000e+00,             nan,
                    nan,  1.00000000e+00],
       [ 0.00000000e+00,  1.47875905e+06,             nan,
                    nan,  1.00000000e+00],
       [ 0.00000000e+00,  1.83599001e+06,             nan,
                    nan,  1.00000000e+00],
       [ 0.00000000e+00,  1.96776694e+06,             nan,
                    nan,  1.00000000e+00],
       [ 0.00000000e+00,  1.63786241e+06,             nan,
                    nan,  1.00000000e+00],
       [ 0.00000000e+00,  1.51068700e+06,             nan,
                    nan,  1.00000000e+00],
       [ 0.00000000e+00,  1.66888447e+06,             nan,
      